In [ ]:
from traffic.core import Traffic
from tqdm.auto import tqdm
import numpy as np
import os

In [ ]:
trajs = Traffic.from_file('data/rectangle_1//combined/combined.parquet')

In [ ]:
trajs = trajs.resample(100).eval(max_workers=20)

In [ ]:
X = []
for flight in tqdm(trajs):
    df = flight.data[['latitude', 'longitude', 'altitude']]
    df = df.interpolate(method='linear', limit_direction='both').ffill().bfill()
    df_as_np = df.to_numpy()
    X.append(df_as_np)

X = np.array(X)

In [ ]:
X.shape

In [ ]:
indexList_X_nan = [np.any(i) for i in np.isnan(X)]
X = np.delete(X, indexList_X_nan, axis=0)
X.shape

### Min Max scaling

In [ ]:
lat_max = np.max(X[:,:,0])
lat_min = np.min(X[:,:,0])
lon_max = np.max(X[:,:,1])
lon_min = np.min(X[:,:,1])
alt_max = np.max(X[:,:,2])
alt_min = np.min(X[:,:,2])
print(lat_max, lat_min, lon_max, lon_min, alt_max, alt_min)

X_norm = X.copy() 
X_norm[:,:,0] = (X_norm[:,:,0] - lat_min) / (lat_max - lat_min)
X_norm[:,:,1] = (X_norm[:,:,1] - lon_min) / (lon_max - lon_min)
X_norm[:,:,2] = (X_norm[:,:,2] - alt_min) / (alt_max - alt_min)

In [ ]:
x=4887

lat = X_norm[x,:,0]
lon = X_norm[x,:,1]
# plot lat/lon
import matplotlib.pyplot as plt
plt.plot(lon, lat)
plt.show()

In [ ]:
if not os.path.exists('data/rectangle_1/training'):
        os.makedirs('data/rectangle_1/training')

np.save('data/rectangle_1/training/X_norm', X_norm)
with open('data/rectangle_1/training/normalisation.txt', 'w') as f:
    f.write(str(lat_max) + ' ' + str(lat_min) + ' ' + str(lon_max) + ' ' + str(lon_min) + ' ' + str(alt_max) + ' ' + str(alt_min))
   

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
latent_dim = 64

encoder_inputs = keras.Input(shape=(100, 3))
x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(25*64, activation="relu")(latent_inputs) # 25 = 100/4 (due to strides=2 in encoder)
x = layers.Reshape((25, 64))(x)
x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv1DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=-1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(X_norm, epochs=100, batch_size=128)

In [ ]:
# function xn that returns a random number between -1 and 1
import random
def xn():
    return random.uniform(-1, 1)

z_sample = np.array([[xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(),
                      xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(),
                      xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(),
                      xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(),
                      xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(),
                      xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(), xn(),
                      xn(), xn(), xn(), xn()]])
vae.decoder.predict(z_sample)

In [ ]:
flight = X_norm[0]
flight

In [ ]:
enc = vae.encoder.predict(flight.reshape(1, 100, 3))
enc[0]

In [ ]:
vae.decoder.predict(enc[0])